# TFA: Time-Frequency Analysis (WIP)

A work in progress for the new interface for TFA, just showing the data access and process application.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime as dt
import numpy as np

from swarmpal.io import create_paldata, PalDataItem
from swarmpal.toolboxes import tfa

## MAGx_LR example

### Fetching data

In [ ]:
data_params = dict(
    collection="SW_OPER_MAGA_LR_1B",
    measurements=["B_NEC"],
    models=["Model='CHAOS-Core'+'CHAOS-Static'"],
    start_time=dt.datetime(2015, 3, 14),
    end_time=dt.datetime(2015, 3, 14, 3, 59, 59),
    server_url="https://vires.services/ows",
    options=dict(asynchronous=False, show_progress=False),
)
data = create_paldata(PalDataItem.from_vires(**data_params))
print(data)

### Applying processes

#### TFA: Preprocess

In [ ]:
p1 = tfa.processes.Preprocess()
p1.set_config(
    dataset="SW_OPER_MAGA_LR_1B",
    active_variable="B_MFA",
    active_component=2,
    remove_model=True,
    convert_to_mfa=True,
)
p1(data)
print(data)

In [ ]:
data["SW_OPER_MAGA_LR_1B"]["B_NEC_res_Model"].plot.line(x="Timestamp");

In [ ]:
data["SW_OPER_MAGA_LR_1B"]["B_MFA"].plot.line(x="Timestamp");

In [ ]:
data["SW_OPER_MAGA_LR_1B"]["TFA_Variable"].plot.line(x="Timestamp");

#### TFA: Clean

In [ ]:
p2 = tfa.processes.Clean()
p2.set_config(
    window_size=300,
    method="iqr",
    multiplier=1,
)
p2(data)
data["SW_OPER_MAGA_LR_1B"]["TFA_Variable"].plot.line(x="Timestamp");

#### TFA: Filter

In [ ]:
p3 = tfa.processes.Filter()
p3.set_config(
    sampling_rate=1,
    cutoff_frequency=20 / 1000,
)
p3(data)
data["SW_OPER_MAGA_LR_1B"]["TFA_Variable"].plot.line(x="Timestamp");

#### TFA: Wavelet

In [ ]:
p4 = tfa.processes.Wavelet()
p4.set_config(
    min_frequency=20 / 1000,
    max_frequency=100 / 1000,
    dj=0.1,
)
p4(data)
print(data)

### Plotting

In [ ]:
tfa.plotting.time_series(data)

In [ ]:
tfa.plotting.spectrum(data, levels=np.linspace(-6, -1, 20))

## MAGx_HR

In [ ]:
data_params = dict(
    collection="SW_OPER_MAGA_HR_1B",
    measurements=["B_NEC"],
    models=["Model='CHAOS-Core'+'CHAOS-Static'"],
    start_time=dt.datetime(2015, 3, 14, 12, 5, 0),
    end_time=dt.datetime(2015, 3, 14, 12, 30, 0),
    server_url="https://vires.services/ows",
    options=dict(asynchronous=False, show_progress=False),
)
data = create_paldata(PalDataItem.from_vires(**data_params))

In [ ]:
p1 = tfa.processes.Preprocess()
p1.set_config(
    dataset="SW_OPER_MAGA_HR_1B",
    active_variable="B_NEC_res_Model",
    active_component=2,
    remove_model=True,
)
p2 = tfa.processes.Clean()
p2.set_config(
    window_size=300,
    method="iqr",
    multiplier=1,
)
p3 = tfa.processes.Filter()
p3.set_config(
    sampling_rate=50,
    cutoff_frequency=0.1,
)
p4 = tfa.processes.Wavelet()
p4.set_config(
    min_frequency=1,
    max_frequency=25,
    dj=0.1,
)
p1(data)
p2(data)
p3(data)
p4(data);

In [ ]:
data

In [ ]:
tfa.plotting.time_series(data)

In [ ]:
tfa.plotting.spectrum(data)

## EFIx_TCT

In [ ]:
data_params = dict(
    collection="SW_EXPT_EFIA_TCT02",
    measurements=["Ehx", "Ehy", "Ehz", "Quality_flags"],
    start_time=dt.datetime(2015, 3, 14, 12, 5, 0),
    end_time=dt.datetime(2015, 3, 14, 12, 30, 0),
    server_url="https://vires.services/ows",
    options=dict(asynchronous=False, show_progress=False),
)
data = create_paldata(PalDataItem.from_vires(**data_params))

In [ ]:
p1 = tfa.processes.Preprocess()
p1.set_config(
    dataset="SW_EXPT_EFIA_TCT02",
    active_variable="Eh_XYZ",
    active_component=2,
)
p2 = tfa.processes.Clean()
p2.set_config(
    window_size=300,
    method="iqr",
    multiplier=1,
)
p3 = tfa.processes.Filter()
p3.set_config(
    sampling_rate=2,
    cutoff_frequency=20 / 1000,
)
p4 = tfa.processes.Wavelet()
p4.set_config(
    min_frequency=20 / 1000,
    max_frequency=200 / 1000,
    dj=0.1,
)
p1(data)
p2(data)
p3(data)
p4(data);

In [ ]:
data

In [ ]:
tfa.plotting.time_series(data)

In [ ]:
tfa.plotting.spectrum(data, levels=np.linspace(-2, 2, 20))